# Tensorflow Object Detection API 视讯测试

In [1]:
# 载入套件
import os
import pathlib
import tensorflow as tf
import pathlib

## GPU 记忆体配置设定

In [2]:
# GPU 设定为 记忆体动态调整 (dynamic memory allocation)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
#  GPU 设定为固定为 2GB 
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*2)])


## 载入模型

In [4]:
# 下载模型，并解压缩
def download_model(model_name, model_date):
    base_url = 'http://download.tensorflow.org/models/object_detection/tf2/'
    model_file = model_name + '.tar.gz'
    # 解压缩
    model_dir = tf.keras.utils.get_file(fname=model_name,
                            origin=base_url + model_date + '/' + model_file,
                            untar=True)
    return str(model_dir)

MODEL_DATE = '20200711'
MODEL_NAME = 'centernet_hg104_1024x1024_coco17_tpu-32'
PATH_TO_MODEL_DIR = download_model(MODEL_NAME, MODEL_DATE)
PATH_TO_MODEL_DIR

1426464768/1426460092 [==============================] - 723s 1us/step


'C:\\Users\\Ariel\\.keras\\datasets\\centernet_hg104_1024x1024_coco17_tpu-32'

## 载入模型

In [5]:
# 快速从下载的目录载入模型
import time
from object_detection.utils import label_map_util, config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

# 组态档及模型档路径
PATH_TO_CFG = PATH_TO_MODEL_DIR + "/pipeline.config"
PATH_TO_CKPT = PATH_TO_MODEL_DIR + "/checkpoint"

# 计时开始
print('Loading model... ', end='')
start_time = time.time()

# 载入组态档，再建置模型
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, 
                                      is_training=False)

# 还原模型
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

# 计时完成
end_time = time.time()
elapsed_time = end_time - start_time
print(f'共花费 {elapsed_time} 秒.')

ModuleNotFoundError: No module named 'object_detection'

## 建立 Label 的对照表

In [ ]:
# 下载 labels file
def download_labels(filename):
    base_url = 'https://raw.githubusercontent.com/tensorflow/models'
    base_url += '/master/research/object_detection/data/'
    label_dir = tf.keras.utils.get_file(fname=filename,
                                        origin=base_url + filename,
                                        untar=False)
    label_dir = pathlib.Path(label_dir)
    return str(label_dir)

LABEL_FILENAME = 'mscoco_label_map.pbtxt'
PATH_TO_LABELS = download_labels(LABEL_FILENAME)
PATH_TO_LABELS

In [ ]:
# 建立 Label 的对照表 (代码与名称)
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## 视讯物件侦测

In [ ]:
@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

In [ ]:
import numpy as np
import cv2

# 使用 webcam
#cap = cv2.VideoCapture(0)

# 读取视讯档案
cap = cv2.VideoCapture('./images_Object_Detection/pedestrians.mp4')
i=0
while True:
    # 读取一帧(frame) from camera or mp4
    ret, image_np = cap.read()

    # 加一维，变为 (笔数, 宽, 高, 颜色)
    image_np_expanded = np.expand_dims(image_np, axis=0)

    # 可测试水平翻转
    # image_np = np.fliplr(image_np).copy()

    # 可测试灰阶
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # 转为 TensorFlow tensor 资料型态
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    
    # detections：物件资讯 内含 (候选框, 类别, 机率)
    detections = detect_fn(input_tensor)    
    num_detections = int(detections.pop('num_detections'))

    # 第一帧(Frame)才显示物件个数
    if i==0:
        print(f'物件个数：{num_detections}')
        
    # 结果存入 detections 变数
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['detection_classes'] = detections['detection_classes'].astype(int)

    # 将物件框起来
    label_id_offset = 1    
    image_np_with_detections = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'] + label_id_offset,
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)

    # 显示侦测结果
    img = cv2.resize(image_np_with_detections, (800, 600))
    cv2.imshow('object detection', img)

    # 存档
    i+=1
    if i==30:
        cv2.imwrite('./images_Object_Detection/pedestrians.png', img)
    
    # 按 q 可以结束
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## 夜晚物件侦测

In [ ]:
import numpy as np
import cv2

# 使用 webcam
#cap = cv2.VideoCapture(0)

# 读取视讯档案
cap = cv2.VideoCapture('./images_Object_Detection/night.mp4')
i=0
while True:
    # 读取一帧(frame) from camera or mp4
    ret, image_np = cap.read()

    # 加一维，变为 (笔数, 宽, 高, 颜色)
    image_np_expanded = np.expand_dims(image_np, axis=0)

    # 可测试水平翻转
    # image_np = np.fliplr(image_np).copy()

    # 可测试灰阶
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # 转为 TensorFlow tensor 资料型态
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    
    # detections：物件资讯 内含 (候选框, 类别, 机率)
    detections = detect_fn(input_tensor)    
    num_detections = int(detections.pop('num_detections'))

    # 第一帧(Frame)才显示物件个数
    if i==0:
        print(f'物件个数：{num_detections}')
        
    # 结果存入 detections 变数
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['detection_classes'] = detections['detection_classes'].astype(int)

    # 将物件框起来
    label_id_offset = 1    
    image_np_with_detections = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'] + label_id_offset,
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)

    # 显示侦测结果
    img = cv2.resize(image_np_with_detections, (800, 600))
    cv2.imshow('object detection', img)

    # 存档
    i+=1
    if i==30:
        cv2.imwrite('./images_Object_Detection/pedestrians.png', img)
    
    # 按 q 可以结束
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# plt.show()
from IPython.display import Image
Image('./images_Object_Detection/night.png')